Homework 2

# Introducation:
    This homework is started by reading the baseline algorithm given by the professor. Implementing the baseline algorithm and then started to read the fifth paper.  Because the third paper is more related to the basic search algorithm, the improvement from the method in the third paper. After finished the paper three improvement, starting to improve the algorithm using the method in the fifth paper.
    



## Beam Search Algorithm with ngram LM:

The baseline is finshed by following the homework 2 description and the 1st and 2nd paper.   

Firstly, writing the basic of it, then adding other components in the order of score(), prune(), ext_limits().
( for winner(), we just use prune() )


For the score(), using score_seq() and score_bitstring() from ngram.py to compute.


## The third paper:
    Calculating sum weight of n-order continous context during our decipherment. This is aim for improving our beam search algorithm in terms of speed. Counting the number of maximum content for each n-gram and sum them up with the weight [1,1,1,1,4,5]. Using that score to decide which cipher symbol to decipher next. However, the result does not seem to have much effect.
    Trying to assign different ext_limit for each letter by the frequency of each letter in the default lm. The improvement is also no effect.



## Neural LM

    At the very beginning, thinking the NLM could be used directly by calling the function Score_first() and Score_next() because difficult understanding the algorithm described in the paper. Our initial understanding is that scanning cipher from left to right. 
    When implementing the algorithm in the fifth paper, finding that our initial understanding was wrong. The paper suggested that use partial hypothesis to decipher in each iteration, which produces a partial decipherment. And using that decipherment and current partial hypothesis to sample in current context. Basically generating sample from left to right. Then implementing the score functin as described in the fifth paper that substracts FMH
    The difficulties we met are:
        1. Did not understand the algorithm well,  so do not understand why having FMH and what's the parameter of FMH
        2. When ciphering the Zodiac-408, llh_predict sometimes gives us punctuations or even spaces, but our assumption is that llg_predict will always return a lowercase letter.
        3. It takes too long to cipher, which makes it harder to try different experiment.
    

# Combine differen approach:

Tring to use nlm score function at the last loop of the beam search algorithm. However, there is almost no effect to the result.
 

## This is our implementation of beam search with n-gram
    (Also having a version of NLM implementation in test_victor.py, but it takes too long)

In [2]:
#====================================================
# test.py
#====================================================
# import operator # tag2vocab()
# import re # countRules()
from auxilaryFunc import *
from ngram import * 
from nlm import * 

from tqdm.autonotebook import tqdm
from collections import Counter
import math # ceil()
import pprint # pprint()
pp = pprint.PrettyPrinter(width=45, compact=True)

# =============================================
# hypo -> dict 
# 		find max score(dict) -> decipher


def sum_of_weight(cipher, weight, taken, cur):
    seq = ''
    taken.append(cur)
    for l in cipher:
        seq += ' ' if l not in taken else 'o'

    words = seq.split()
    list_ngram = {}
    for w in words:
        if len(w) not in list_ngram:
            list_ngram[len(w)] = 1
        else:
            list_ngram[len(w)] += 1
    score = 0
    for k in list_ngram:
        w = weight[k-1] if k <= 6 else weight[-1]
        score += w*list_ngram[k]
    return  score


# ext_order by frequency
def getExtOrd(cipher):
	freq = sorted(Counter(cipher).items(), key=lambda k:k[1], reverse=True)
	return tuple([ x[0] for x in freq ])


# get mask
def seq2bitstring(seq):
	bitstring = ''
	for i in seq:
		bitstring += 'o' if i != '_' else '.'
	return bitstring 

# convert hypo to dict
def readHypo(h):  
	dict = {}
	for rule in h:
		dict[rule[1]]= rule[0]
	return dict

# get winner
def decipher(cipher, dict):
	decipher = ''
	for l in cipher:
		decipher += '' if l not in dict else dict[l]
	return decipher

# compute score with dict & cipher
def score(cipher, dict, nlm=False):
	seq = ''
	for l in cipher:
		seq += '_' if l not in dict else dict[l]

	if nlm == True: 
		return score_sequence(seq, nLM, cuda=True)
	else:
		return lm.score_bitstring(seq, seq2bitstring(seq)) #- FMH('B','BURGER','b',plaintext)

def score_nlm(cipher, dict):
	seq = ''
	for l in cipher:
		seq += '\t' if l not in dict else dict[l]
	# print(seq)

def hypo2score(cipher, h, nlm=False):
	return score(cipher, readHypo(h), nlm)

def checkLimit(h, ext_limits):
	letters = list(map( lambda x : x[0], h))
	dict = {}
	for k in letters:
		if k not in dict:
		    dict[k] = 1
		else:
		    dict[k] += 1
		    if dict[k] > ext_limits:
		    	return False
	return True

def checkLimit_arr(h, ext_limits):
	letters = list(map( lambda x : x[0], h))
	dict = {}
	for k in letters:
		if k not in dict:
		    dict[k] = 1
		else:
		    dict[k] += 1
		    if dict[k] > ext_limits[k]:
		    	return False
	return True

def get_ext_limit_arr(length, letter, freq_perc):
	dict = {}

	for l, p in zip(letter, freq_perc):
		dict[l] = (p+1)*length
	return dict

# get score of all hypos
def sortScoreTable(cipher, hypos, nlm=False):
	score_table = []

	if nlm == False:
		for i in range(len(hypos)):
			score_table.append([ hypo2score(cipher, hypos[i], nlm), hypos[i] ]) 
	else:
		for i in tqdm(range(len(hypos))):
			score_table.append([ hypo2score(cipher, hypos[i], nlm), hypos[i] ]) 

	return sorted(score_table, key=lambda x: x[0], reverse=True)

# prune sorted score by beam_size
def prune(cipher, hypos, beam_size, nlm=False):
	sorted_table = sortScoreTable(cipher, hypos, nlm)
	prune_size = beam_size if beam_size < len(sorted_table) else len(sorted_table)
	return tuple([ x[1] for x in sorted_table[:prune_size] ]), sorted_table[0][0]


def checkAnswer(filename, dec): 
	# gold_file = "data/_ref.txt"
	gold_file = filename
	ser = symbol_error_rate(dec, gold_file)
	print('Error: ', ser*100, 'Accuracy: ', (1-ser)*100)

def beamSearch(cipher, beam_size, ext_limits, answer, nlm=False):
	letter = ('a', 'b', 'c', 'd', 'e', 'f', 'g',
			 'h', 'i', 'j', 'k', 'l', 'm', 'n',
			  'o', 'p', 'q', 'r', 's', 't', 'u',
			   'v', 'w', 'x', 'y', 'z')


	cipher_clean = ''.join(cipher.replace('\n', '').split())
	ext_ord = getExtOrd(cipher_clean)
	print('EXT_ORDER:', ext_ord)

	if ext_limits != 0:
		min_limit = math.ceil(len(ext_ord) / len(letter))
		if ext_limits < min_limit: ext_limits = min_limit
	

	freq_perc = [0.05714285714285714, 0.02857142857142857, 0.02857142857142857, 
    0.02857142857142857, 0.11428571428571428, 0.02857142857142857, 0.02857142857142857, 
    0.02857142857142857, 0.05714285714285714, 0.02857142857142857, 0.02857142857142857, 
    0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 
    0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 
    0.05714285714285714, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 
    0.02857142857142857, 0.02857142857142857, 0.02857142857142857]
# 	ext_limits_arr = {'e': 7, 'a': 5, 'i': 4, 'n': 4,
# 	 'o': 4, 't': 4, 'l': 3, 's': 3, 'r': 3, 'h': 2,
# 	  'f': 2, 'd': 2, 'k': 1, 'g': 1, 'p': 1, 'b': 1,
# 	   'c': 1, 'u': 1, 'm': 1, 'w': 1, 'v': 1, 'x': 1, 'y': 1}

	length = len(ext_ord)
	ext_limits_arr = get_ext_limit_arr(length, letter, freq_perc)

	hs, ht = ((),) , []

	# --------------------------------
	#	
	# ext_ord = list(ext_ord)
	# taken = []
	# weight_matrix = [1,1,1,1,4,5]
	# --------------------------------	

	for card in tqdm(range(length)):

	# --------------------------------	
	#
		# f = ext_ord[0] #if len(ext_ord) >1 else ext_ord[0] 
		# taken.append(f)
		# ext_ord.remove(f)

	# --------------------------------	
		
		f = ext_ord[card]
	# --------------------------------	
		for h in hs: 
			for e in letter:
				if h == ():
					ht.append((( e, f ),)) 
				else:
					h_ = h + (( e, f ),)
					if ext_limits != 0:
						if not checkLimit(h_, ext_limits):
							continue
					else:
						if not checkLimit_arr(h_, ext_limits_arr):
							continue
					ht.append( h_ ) 
		hs, _ = prune(cipher_clean, ht, beam_size)
		ht = []
		# print(hs)

	# --------------------------------	
	#
		# current_sum = 0
		# sum_max = 0
		# for c in ext_ord:
		# 	current_sum = sum_of_weight(cipher, weight_matrix, taken, c)
		# 	if current_sum >= sum_max:
		# 		f = c
		# 		sum_max = current_sum
	# --------------------------------	

	hs, score = prune(cipher_clean, hs, beam_size, nlm)

	print('score:',score)
	pp.pprint(hs[0])
	dict = readHypo(hs[0])
	dec = decipher(cipher, dict)
	print(dec)
	#checkAnswer(answer, dec)

# =============================================
# main

if __name__ == '__main__':
	lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

	# cipher = 'INTERNATIONALSTUDENTSOCIETY'
	cipher = 'F7EZ5F UC2'
	cipher = 'efgfoe uif fbtu xbmm pg uif dbtumf'
	cipher = 'F7EZ5F UC2 1DR6 M9PP 0E 6CZ SD4UP1'
	# cipher = 'RIHILKHIERSKILEKKLKTRSKHRIHILKHLREIRTRSKLKTRSKHHLEKRS'
	cipher = read_file("data/cipher.txt")

	answer = 'data/_ref.txt'
	# answer = 'data/answer.txt'
	
	ext_limits = 1 # 0 for using limit_arr
	beam_size = 3000
	nlm = False
	ext_ord_dynamic = True

	if nlm: 
		nLM = load_model("data/mlstm_ns.pt", cuda=True)
	beamSearch(cipher, beam_size, ext_limits, answer, nlm)


# =============================================
# =============================================
# =============================================

Reading language model from data/6-gram-wiki-char.lm.bz2...


EXT_ORDER: ('—', 'º', 'B', 'R', '∫', 'P', 'I', 'u', '≈', 'y', '∑', 'À', 'X', 'W', 'V', 'E', 'Z', '+', '∞', 'H', 'M', 'µ', '£', 'A', 'L', 'æ', '∏', 'O', '•', 'G', 'T', '‘', '“', '/', '–', 'F', 'π', 'J', '^', 'N', 'D', 'S', 'ƒ', '√', 'K', 'Ç', 'Q', 'Ã', '\\', '¢', '§', 'Ω', '∆', 'j')


Done.



score: -642.8312139635002
(('e', '—'), ('a', 'º'), ('n', 'B'),
 ('e', 'R'), ('t', '∫'), ('t', 'P'),
 ('e', 'I'), ('r', 'u'), ('a', '≈'),
 ('r', 'y'), ('i', '∑'), ('a', 'À'),
 ('t', 'X'), ('n', 'W'), ('o', 'V'),
 ('r', 'E'), ('h', 'Z'), ('s', '+'),
 ('n', '∞'), ('i', 'H'), ('c', 'M'),
 ('s', 'µ'), ('s', '£'), ('i', 'A'),
 ('o', 'L'), ('l', 'æ'), ('d', '∏'),
 ('h', 'O'), ('c', '•'), ('u', 'G'),
 ('u', 'T'), ('g', '‘'), ('h', '“'),
 ('o', '/'), ('d', '–'), ('l', 'F'),
 ('u', 'π'), ('d', 'J'), ('g', '^'),
 ('w', 'N'), ('m', 'D'), ('m', 'S'),
 ('b', 'ƒ'), ('c', '√'), ('m', 'K'),
 ('l', 'Ç'), ('y', 'Q'), ('k', 'Ã'),
 ('y', '\\'), ('j', '¢'), ('g', '§'),
 ('v', 'Ω'), ('k', '∆'), ('q', 'j'))
attohorntahecdtcnnosaurlnaitumleracdrgreaveugwyrmssmjoaintheirtkekeragochdbhydlionalruseumamesnetlsgmjewgerrkhleunuyminobotinutericrgeeachmedesnellsochaintmesqcnywjarrialhcdhonessgoskgenieamedurhybaoutdyitsclergtoswormicuheldasalockwishjtgrdaiainonysoughtgcmhkkranvmguttecwasmcroatthkitinodatenedlilcsuaian